# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv
wpy_data = pd.read_csv("../output_data/cities.csv")
wpy_data.dropna()
del wpy_data["Unnamed: 0"]
wpy_data.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,pavlovsk,73,RU,1580712652,91,50.46,40.11,36.37,12.97
1,punta arenas,90,CL,1580712653,81,-53.15,-70.92,50.00,9.17
2,atuona,0,PF,1580712653,76,-9.80,-139.03,81.72,15.46
3,mataura,100,NZ,1580712653,96,-46.19,168.86,60.01,1.99
4,saint-philippe,18,RE,1580712653,79,-21.36,55.77,84.99,27.51


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store Lat and Lng
locations = wpy_data[["Lat", "Lng"]]
humidity = wpy_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Create new Dataframe
max_temp = wpy_data[wpy_data["Max Temp"] < 80]
min_temp = max_temp[max_temp["Max Temp"] > 60]
windy = min_temp[min_temp["Wind Speed"] < 10]
cloudy = windy[windy["Cloudiness"] == 0]
humid = cloudy[cloudy["Humidity"] < 60]
humid

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
121,wukari,0,NG,1580712675,21,7.85,9.78,65.68,5.88
122,yaan,0,NG,1580712675,19,7.38,8.57,67.57,4.70
199,los llanos de aridane,0,ES,1580712688,36,28.66,-17.92,69.01,5.82
208,general roca,0,AR,1580712632,33,-39.03,-67.58,71.60,4.70
242,nouadhibou,0,MR,1580712696,45,20.93,-17.03,64.40,9.17
293,tsumeb,0,NaN,1580712706,53,-19.23,17.72,73.40,5.82
300,santiago del estero,0,AR,1580712708,58,-27.80,-64.26,75.99,4.52
302,willowmore,0,ZA,1580712708,45,-33.29,23.49,77.61,1.99
340,fort abbas,0,PK,1580712715,28,29.19,72.85,64.90,5.97
351,san martin,0,AR,1580712702,50,-33.08,-68.47,78.80,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Set empty list
hotel_df = []

# Add hotel to data frame
hotels = humid.reset_index(drop=True)
hotels["Hotel Name"] = ""

In [7]:
# Set url for API
g_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Loop through each city
for city in range(len(hotels["City"])):
    
    lat = hotels.loc[city]["Lat"]
    lon = hotels.loc[city]["Lng"]
    params = {
        "location": f"{lat},{lon}",
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key
    }

    # Create search query, make request and store in json
    query = requests.get(g_url, params)
    query_json = query.json()
        
    # Grab hotel name
    try:
        hotel_df.append(query_json["results"][0]["name"])
    except:
        hotel_df.append("No hotel found")
        pass

hotels["Hotel Name"] = hotel_df
hotels

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,wukari,0,NG,1580712675,21,7.85,9.78,65.68,5.88,No hotel found
1,yaan,0,NG,1580712675,19,7.38,8.57,67.57,4.70,No hotel found
2,los llanos de aridane,0,ES,1580712688,36,28.66,-17.92,69.01,5.82,No hotel found
3,general roca,0,AR,1580712632,33,-39.03,-67.58,71.60,4.70,No hotel found
4,nouadhibou,0,MR,1580712696,45,20.93,-17.03,64.40,9.17,No hotel found
5,tsumeb,0,NaN,1580712706,53,-19.23,17.72,73.40,5.82,No hotel found
6,santiago del estero,0,AR,1580712708,58,-27.80,-64.26,75.99,4.52,No hotel found
7,willowmore,0,ZA,1580712708,45,-33.29,23.49,77.61,1.99,No hotel found
8,fort abbas,0,PK,1580712715,28,29.19,72.85,64.90,5.97,No hotel found
9,san martin,0,AR,1580712702,50,-33.08,-68.47,78.80,3.36,No hotel found


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map